In [1]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



In [2]:
X_data, y_data = hete.load_xy(subsample=50)

X shape: (50, 64, 64, 61, 4) | y shape: (50, 64, 64, 61, 2)


***
# END

In [9]:
x = np.moveaxis(np.moveaxis(X_data, -2, 1).reshape(X_data.shape[0]*61,64,64,4), -1, 1)
y = np.moveaxis(np.moveaxis(y_data, -2, 1).reshape(X_data.shape[0]*61,64,64,2), -1, 1)
print(x.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
print('X_train: {} | y_train: {}\nX_test:  {} | y_test:  {}'.format(X_train.shape, y_train.shape, X_test.shape,  y_test.shape))

(3050, 4, 64, 64) (3050, 2, 64, 64)
X_train: (2043, 4, 64, 64) | y_train: (2043, 2, 64, 64)
X_test:  (1007, 4, 64, 64) | y_test:  (1007, 2, 64, 64)


In [ ]:
device = 'cuda'

rom = h2_hete_rom().to(device)
optimizer = Adam(rom.parameters(), lr=1e-3, weight_decay=1e-6)
loss_fn = nn.MSELoss()

loss, val_loss = [], []
metrics = {'loss':[], 'val_loss':[]}
epochs = 50
batch_size = 100

for epoch in range(epochs):
    rom.train()
    epoch_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        inp  = torch.Tensor(X_train[i:i+batch_size]).to(device)
        true = torch.Tensor(y_train[i:i+batch_size]).to(device)
        optimizer.zero_grad()
        pred = rom(inp)
        loss = loss_fn(pred,true)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()*inp.size(0)
    metrics['loss'].append(epoch_loss/len(X_train))
    
plt.figure(figsize=(4,3))
plt.plot(metrics['loss'], label='loss')
plt.legend(); plt.grid('on')
plt.title('Training Loss'); plt.xlabel('epoch'); plt.ylabel('loss')
plt.show()

In [ ]:
vis_realization = 411

x_true = np.moveaxis(np.moveaxis(X_data[vis_realization], -2, 0), -1, 1)
y_true = np.moveaxis(np.moveaxis(y_data[vis_realization], -2, 0), -1, 1)
print(x_true.shape, y_true.shape)

y_pred = rom(torch.Tensor(x_true).to(device)).cpu().detach().numpy()
print(y_pred.shape)

labels = ['$\phi$', '$P$', '$\hat{P}$', '$|P-\hat{P}|$', '$S$', '$\hat{S}$', '$|S-\hat{S}|$']
fig, axs = plt.subplots(7, 12, figsize=(20,8))
for j in range(12):   
    axs[0,j].imshow(x_true[-1,0], 'jet'); axs[0,j].set(title='R{} t{}'.format(vis_realization, j*5))
    p,  s  = y_true[j*5,0], y_true[j*5,1]
    p_, s_ = y_pred[j*5,0], y_pred[j*5,1]
    axs[1,j].imshow(p, 'jet')               ; axs[4,j].imshow(s, 'jet')
    axs[2,j].imshow(p_, 'jet')              ; axs[5,j].imshow(s_, 'jet')
    axs[3,j].imshow(np.abs(p-p_), 'binary') ; axs[6,j].imshow(np.abs(s-s_), 'binary')
    for i in range(7):
        axs[i,j].set(xticks=[], yticks=[])
for i in range(7):
    axs[i,0].set(ylabel=labels[i])
plt.show() 